# Pandas 데모 - Explarotary Data Analysis

## 라이브러리 import 및 설정

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler, RobustScaler # 데이터 표준화

# lightgbm 모델
from lightgbm import LGBMClassifier, plot_importance

# normalization
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

#feature Selection
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import itertools

In [4]:
rcParams['figure.figsize'] = (18, 9)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

##  필요한 함수 선언

In [5]:
def Standardzation_df(df):
    X = RobustScaler().fit_transform(df)
    return pd.DataFrame(X)

In [6]:
def Normalization_df(df):
    X = MinMaxScaler().fit_transform(df)
    return pd.DataFrame(X)

# Main 함수

## 데이터 다운로드

데이터는 [Dacon 단국대 소/중 데이터 분석 AI 경진대회 웹사이트](https://www.dacon.io/competitions/official/235638/data/)에서 다운로드 받아 `../input` 폴더에 저장.

In [7]:
!ls -alF ../input/

total 693784
drwxr-xr-x 2 swcu swcu      4096 Sep 30 03:46 ./
drwxr-xr-x 8 swcu swcu      4096 Sep 30 04:41 ../
-rw-rw-r-- 1 swcu swcu    800010 Jul 26 04:35 sample_submission.csv
-rw-r--r-- 1 swcu swcu  57325897 Sep 30 03:48 scaled_test.csv
-rw-r--r-- 1 swcu swcu 557070064 Sep 30 03:48 scaled_trn.csv
-rw-rw-r-- 1 swcu swcu  18936246 Jul 26 04:35 test.csv
-rw-rw-r-- 1 swcu swcu  76278443 Jul 26 04:35 train.csv


In [8]:
data_dir = Path('../input/')
trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
# feature_file = data_dir / 'feature.csv'
sample_file = data_dir / 'sample_submission.csv'
submission_dir = Path('../submission')
target_col = 'class'
seed = 42

In [9]:
algo_name = 'lgb'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

#feature_file = feature_dir / f'{feature_name}.csv'
submission_file = submission_dir / f'{model_name}.csv'

## EDA

In [10]:
trn = pd.read_csv(trn_file, index_col=0)
print(trn.shape)
trn.head()

(320000, 19)


,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0


## 데이터 개요

In [11]:
trn.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,3.2000e+05,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000
mean,19.8524,18.3768,17.7358,17.1742,16.8752,5.8308e-02,19.6289,18.2271,17.6154,17.1523,16.8693,6.3327,6.1388,1.1758,1.1765,1.1751,1.1754,1.1761,1.1163
std,1.9455,31.1108,1.4662,52.5615,57.1463,5.7546e-01,1.9236,17.7866,1.4730,25.0758,35.4365,8.8811,8.5516,0.1163,0.1181,0.1147,0.1155,0.1172,0.9235
min,-17.6875,-17555.3656,7.7314,-23955.8178,-23955.8178,-1.6605e+02,-42.8375,-9999.0000,-16.3802,-9999.0000,-9999.0000,1.0000,1.0000,1.0001,1.0000,1.0002,1.0001,1.0000,0.0000
25%,18.7242,17.4749,16.8772,16.5239,16.2889,3.5580e-05,18.5636,17.3484,16.7871,16.4530,16.2341,1.0000,1.0000,1.0882,1.0884,1.0878,1.0880,1.0883,0.0000
50%,19.4194,18.1405,17.5258,17.1497,16.9176,4.7129e-02,19.2646,18.0224,17.4434,17.0874,16.8694,2.0000,2.0000,1.1793,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4321,19.0732,18.4281,18.0074,17.7289,9.4597e-02,20.1976,18.8884,18.2911,17.9070,17.6557,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,58.6233,46.3383,72.0097,50.1641,52.6127,6.2323e+01,30.7779,30.6132,31.9572,32.0162,30.7813,44.0000,42.0000,2.0508,2.0803,2.0221,2.0363,2.0654,2.0000


In [12]:
trn.dtypes

u            float64
g            float64
r            float64
i            float64
z            float64
redshift     float64
dered_u      float64
dered_g      float64
dered_r      float64
dered_i      float64
dered_z      float64
nObserve       int64
nDetect        int64
airmass_u    float64
airmass_g    float64
airmass_r    float64
airmass_i    float64
airmass_z    float64
class          int64
dtype: object

In [13]:
for i in trn.columns:
    print(i, len(trn.loc[trn[i]<0]))


u 3
g 4
r 0
i 4
z 9
redshift 70876
dered_u 8
dered_g 9
dered_r 6
dered_i 7
dered_z 8
nObserve 0
nDetect 0
airmass_u 0
airmass_g 0
airmass_r 0
airmass_i 0
airmass_z 0
class 0


In [14]:
col = ['u', 'g', 'r', 'i', 'z', 'dered_u', 'dered_g', 
      'dered_r', 'dered_i', 'dered_z']
trn1 = trn.copy()
for i in col:
    trn1 = trn1[trn1[i] >=0]

trn1

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319995,18.3656,17.2409,16.7739,16.4850,16.3404,5.1988e-02,18.1328,17.0406,16.6374,16.3861,16.2874,2,2,1.0930,1.0888,1.0973,1.0951,1.0909,2
319996,18.6856,17.6658,17.1904,16.8169,16.6367,8.7261e-02,18.5207,17.5131,17.0871,16.7436,16.6017,2,2,1.0517,1.0508,1.0526,1.0522,1.0513,1
319997,21.1890,18.4234,17.2956,16.9011,16.6841,-1.0148e-05,20.9890,18.3076,17.2152,16.8403,16.6355,31,30,1.1933,1.1941,1.1925,1.1929,1.1937,0


In [15]:
trn1.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,3.1998e+05,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000
mean,19.8528,18.4320,17.7358,17.3043,17.0613,5.8309e-02,19.6301,18.2593,17.6161,17.2154,16.9947,6.3328,6.1390,1.1758,1.1765,1.1751,1.1754,1.1761,1.1163
std,1.9428,1.6591,1.4662,1.3198,1.3333,5.7548e-01,1.9072,1.6566,1.4665,1.3249,1.3233,8.8814,8.5519,0.1163,0.1181,0.1147,0.1155,0.1172,0.9235
min,2.2651,1.2383,7.7314,2.7492,5.1281,-1.6605e+02,0.7538,3.0828,6.0403,6.8561,7.2567,1.0000,1.0000,1.0001,1.0000,1.0002,1.0001,1.0000,0.0000
25%,18.7242,17.4749,16.8772,16.5239,16.2890,3.5580e-05,18.5637,17.3484,16.7873,16.4531,16.2342,1.0000,1.0000,1.0882,1.0884,1.0878,1.0880,1.0883,0.0000
50%,19.4195,18.1405,17.5258,17.1498,16.9177,4.7133e-02,19.2646,18.0224,17.4434,17.0874,16.8694,2.0000,2.0000,1.1793,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4320,19.0729,18.4281,18.0075,17.7290,9.4600e-02,20.1976,18.8885,18.2912,17.9070,17.6556,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,58.6233,46.3383,72.0097,50.1641,52.6127,6.2323e+01,30.7779,30.6132,31.9572,32.0162,30.7813,44.0000,42.0000,2.0508,2.0803,2.0221,2.0363,2.0654,2.0000


In [16]:
tst = pd.read_csv(tst_file, index_col = 0)
tst

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z
id,,,,,,,,,,,,,,,,,,
320000,19.1431,18.0172,17.4733,17.1144,16.9263,1.2130e-01,19.0750,17.9550,17.4308,17.0838,16.9087,1,1,1.1563,1.1515,1.1613,1.1588,1.1539
320001,18.7409,17.1705,16.3691,15.9865,15.7307,-6.6907e-05,15.0703,14.3098,14.3901,14.5160,14.6372,2,2,1.0076,1.0074,1.0080,1.0078,1.0075
320002,19.2980,17.8803,17.2221,16.8096,16.6352,1.1001e-01,19.1004,17.7726,17.1460,16.7497,16.5689,1,1,1.3809,1.3905,1.3715,1.3761,1.3856
320003,18.0775,17.1569,16.9041,16.8118,16.7955,2.9697e-05,17.9729,17.0735,16.8470,16.7700,16.7668,1,1,1.1408,1.1408,1.1409,1.1408,1.1408
320004,17.8325,16.7646,16.2135,15.8722,15.6621,6.8471e-02,17.7545,16.6794,16.1563,15.8315,15.6433,2,2,1.1110,1.1077,1.1145,1.1127,1.1093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,19.6734,18.6617,18.2708,18.1225,18.0627,-3.1726e-04,19.3361,18.3933,18.0860,17.9861,17.9670,17,17,1.1858,1.1863,1.1855,1.1857,1.1861
399996,21.5229,20.7283,20.6087,21.0049,20.8458,6.5455e-02,21.3993,20.6399,20.5451,20.9507,20.7827,8,4,1.2850,1.2908,1.2793,1.2821,1.2878
399997,19.3991,18.2411,18.1605,18.1978,18.2018,-8.4180e-04,18.9136,17.8753,17.9047,18.0043,18.0400,1,1,1.1885,1.1886,1.1885,1.1885,1.1885


In [17]:
df = pd.concat([trn1, tst])
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,19.6734,18.6617,18.2708,18.1225,18.0627,-3.1726e-04,19.3361,18.3933,18.0860,17.9861,17.9670,17,17,1.1858,1.1863,1.1855,1.1857,1.1861,NaN
399996,21.5229,20.7283,20.6087,21.0049,20.8458,6.5455e-02,21.3993,20.6399,20.5451,20.9507,20.7827,8,4,1.2850,1.2908,1.2793,1.2821,1.2878,NaN
399997,19.3991,18.2411,18.1605,18.1978,18.2018,-8.4180e-04,18.9136,17.8753,17.9047,18.0043,18.0400,1,1,1.1885,1.1886,1.1885,1.1885,1.1885,NaN


# Feature Engineering

In [18]:
target_col = 'class'
feature_col = [x for x in trn1.columns if trn1[x].dtype in [np.int64, np.float64] and x !=target_col]
len(feature_col)

18

In [19]:
oneChar_col = ['u','g','r','i','z']
dered_col = [c for c in df.columns if c.find('dered') != -1]
airmass_col = [c for c in df.columns if c.find('airmass') != -1]
print(dered_col)
print(airmass_col)

['dered_u', 'dered_g', 'dered_r', 'dered_i', 'dered_z']
['airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z']


In [20]:
#모든 조합 차이 feature
diff_feature = []
for c1, c2 in itertools.combinations(feature_col,2):
    new_c = f'{c1}_{c2}_diff'
    df[new_c] = df[c1] - df[c2]
    diff_feature.append(new_c)
# 모든 조합 나누기    
divide_feature = []
for c1, c2 in itertools.combinations(feature_col,2):
    new_c = f'{c1}_{c2}_divide'
    df[new_c] = df[c1] / df[c2]
    divide_feature.append(new_c)

df.columns.shape

(325,)

## Adding PCA 

In [21]:
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA



In [22]:
#default pca - 
temp = df.columns
temp = temp.drop('class')
pca = PCA(n_components=0.95)# 주성분 개수 설정 
X_pca = pca.fit_transform(df[temp])
print('eigen_value :', pca.explained_variance_)
print('explained variance ratio :', pca.explained_variance_ratio_)


df['PCA'] = X_pca
df['PCA'].shape

eigen_value : [1.11660542e+16]
explained variance ratio : [0.99948469]


(399978,)

In [23]:
X_pca.shape

(399978, 1)

In [24]:
# incrementalPCA - difference is just dividing n_batches for memory 
n_batches = 100
inc_pca = IncrementalPCA(n_components=18)#temp.size*0.5)
for batch_x in np.array_split(df[temp], n_batches):
    print(".", end="") # not shown in the book
    inc_pca.partial_fit(batch_x)
    
X_inc = inc_pca.transform(df[temp])


# print('eigen_value :', inc_pca.explained_variance_)
# print('explained variance ratio :', inc_pca.explained_variance_ratio_)

# X_inc

X_inc_t = X_inc.transpose()

....................................................................................................

In [25]:
for i , ele in enumerate(X_inc_t):
    incPCAn = 'incPCA'+str(i)
    df[incPCAn] = ele
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,...,dered_i_airmass_r_divide,dered_i_airmass_i_divide,dered_i_airmass_z_divide,dered_z_nObserve_divide,dered_z_nDetect_divide,dered_z_airmass_u_divide,dered_z_airmass_g_divide,dered_z_airmass_r_divide,dered_z_airmass_i_divide,dered_z_airmass_z_divide,nObserve_nDetect_divide,nObserve_airmass_u_divide,nObserve_airmass_g_divide,nObserve_airmass_r_divide,nObserve_airmass_i_divide,nObserve_airmass_z_divide,nDetect_airmass_u_divide,nDetect_airmass_g_divide,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,...,14.8296,14.8249,14.8143,0.9394,0.9394,14.2120,14.2011,14.2214,14.2169,14.2067,1.0000,15.1291,15.1174,15.1390,15.1343,15.1234,15.1291,15.1174,15.1390,15.1343,15.1234,0.9992,1.0007,1.0003,0.9996,1.0014,1.0011,1.0004,0.9997,0.9990,0.9993,4.1012e+05,-4.1012e+05,-44588.1645,700.0365,-705.3298,-2043.2987,3.8336,-80.0839,-3.4657,17.4298,12.8590,6.3963,-1.8975,0.1039,-1.1943,-0.1168,-0.3289,-1.2392,0.8164
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,...,10.5995,10.5806,10.5423,13.1347,13.1347,10.4805,10.4423,10.5182,10.4994,10.4614,1.0000,0.7979,0.7950,0.8008,0.7994,0.7965,0.7979,0.7950,0.8008,0.7994,0.7965,0.9964,1.0036,1.0018,0.9982,1.0073,1.0055,1.0018,0.9982,0.9946,0.9964,-1.3453e+05,1.3453e+05,1329.3993,234.1073,-139.3329,141.9778,0.8354,26.8378,4.3943,-49.9491,3.9271,6.1358,0.9277,-2.3727,0.1881,-0.6354,-0.4376,1.1678,-0.0957
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,...,15.0068,14.9959,14.9730,7.6480,7.6480,14.9595,14.9360,14.9817,14.9708,14.9479,1.0000,1.9560,1.9529,1.9589,1.9575,1.9545,1.9560,1.9529,1.9589,1.9575,1.9545,0.9984,1.0015,1.0008,0.9992,1.0031,1.0023,1.0008,0.9993,0.9977,0.9985,-2.0247e+05,2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-0.3252,15.6501,-15.6121,-13.6220,-9.5625,8.5752,0.3751,-0.9788,0.0770,-0.7371,0.0022,0.2375,0.3560
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,

In [26]:
# zip 함수를 이용한 row별, 시리즈 그룹별 max, min, max-min, std, sum 을 feature로 생성
# 그 외에도 mean, skew 등을 사용해 볼 수도 있긴 함 

for prefix, g in zip(['one','dered','airmass'], [oneChar_col, dered_col, airmass_col]):
    
    df[f'{prefix}_max'] = df[g].max(axis=1)
    
    df[f'{prefix}_min'] = df[g].min(axis=1)
    
    df[f'{prefix}_diff'] = df[f'{prefix}_max'] - df[f'{prefix}_min']
    
    df[f'{prefix}_std'] = df[g].std(axis=1)
    
    df[f'{prefix}_sum'] = df[g].sum(axis=1)
    
    

In [27]:
# 각 그룹별 max-max, min-min, sum-sum, std-std을 feature로 생성

for c in itertools.combinations(['one','dered','airmass'],2):
    df[f'{c[0]}_{c[1]}_max_diff'] = df[f'{c[0]}_max'] - df[f'{c[1]}_max']
    
    df[f'{c[0]}_{c[1]}_min_diff'] = df[f'{c[0]}_min'] - df[f'{c[1]}_min']
    
    df[f'{c[0]}_{c[1]}_sum_diff'] = df[f'{c[0]}_sum'] - df[f'{c[1]}_sum']
    
    df[f'{c[0]}_{c[1]}_std_diff'] = df[f'{c[0]}_std'] - df[f'{c[1]}_std']

In [28]:
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,...,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,...,1.0004,0.9997,0.9990,0.9993,4.1012e+05,-4.1012e+05,-44588.1645,700.0365,-705.3298,-2043.2987,3.8336,-80.0839,-3.4657,17.4298,12.8590,6.3963,-1.8975,0.1039,-1.1943,-0.1168,-0.3289,-1.2392,0.8164,23.2640,16.9396,6.3243,2.4994,97.2223,23.1243,16.9089,6.2153,2.4562,96.8782,1.1907,1.1890,1.7020e-03,6.7348e-04,5.9490,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,...,1.0018,0.9982,0.9946,0.9964,-1.3453e+05,1.3453e+05,1329.3993,234.1073,-139.3329,141.9778,0.8354,26.8378,4.3943,-49.9491,3.9271,6.1358,0.9277,-2.3727,0.1881,-0.6354,-0.4376,1.1678,-0.0957,15.0521,13.1689,1.8832,0.7807,69.0039,14.9664,13.1347,1.8317,0.7586,68.7533,1.2578,1.2488,9.0740e-03,3.5867e-03,6.2664,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,...,1.0008,0.9993,0.9977,0.9985,-2.0247e+05,2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-0.3252,15.6501,-15.6121,-13.6220,-9.5625,8.5752,0.3751,-0.9788,0.0770,-0.7371,0.0022,0.2375,0.3560,16.7864,15.3500,1.4364,0.5934,78.8917,16.6076,15.2961,1.3116,0.5462,78.3521,1.0241,1.0210,3.1220e-03,1.2343e-03,5.1126,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0,4.4719,5.4395,5.7657,6.0260,25.6606,0.3070,4.6660,5.5734,5.8660,6.1055,21.6606,22.6606,24.4552,24.4546,24.4558,24.4555,24.4549,0.9676,1.2938,1.5541,21.1887,-4.1649,0.1941,1.1015,1.3941,1.6335,17.1887,

In [29]:
# class 순서 변경
y = df['class']
df.drop('class', inplace = True, axis= 1)
df['class'] = y.values
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,g_airmass_i_diff,...,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff,class
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,19.1474,...,0.9997,0.9990,0.9993,4.1012e+05,-4.1012e+05,-44588.1645,700.0365,-705.3298,-2043.2987,3.8336,-80.0839,-3.4657,17.4298,12.8590,6.3963,-1.8975,0.1039,-1.1943,-0.1168,-0.3289,-1.2392,0.8164,23.2640,16.9396,6.3243,2.4994,97.2223,23.1243,16.9089,6.2153,2.4562,96.8782,1.1907,1.1890,1.7020e-03,6.7348e-04,5.9490,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555,0.0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,12.8110,...,0.9982,0.9946,0.9964,-1.3453e+05,1.3453e+05,1329.3993,234.1073,-139.3329,141.9778,0.8354,26.8378,4.3943,-49.9491,3.9271,6.1358,0.9277,-2.3727,0.1881,-0.6354,-0.4376,1.1678,-0.0957,15.0521,13.1689,1.8832,0.7807,69.0039,14.9664,13.1347,1.8317,0.7586,68.7533,1.2578,1.2488,9.0740e-03,3.5867e-03,6.2664,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550,1.0
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,14.8037,...,0.9993,0.9977,0.9985,-2.0247e+05,2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-0.3252,15.6501,-15.6121,-13.6220,-9.5625,8.5752,0.3751,-0.9788,0.0770,-0.7371,0.0022,0.2375,0.3560,16.7864,15.3500,1.4364,0.5934,78.8917,16.6076,15.2961,1.3116,0.5462,78.3521,1.0241,1.0210,3.1220e-03,1.2343e-03,5.1126,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450,0.0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,4.4719,5.4395,5.7657,6.0260,25.6606,0.3070,4.6660,5.5734,5.8660,6.1055,21.6606,22.6606,24.4552,24.4546,24.4558,24.4555,24.4549,0.9676,1.2938,1.5541,21.1887,-4.1649,0.1941,1.1015,1.3941,1.6335,17.1887,18.1887,19.

# 표준화 및 정규화

In [30]:
y = df['class']
df_X = df.drop('class', axis = 1)
df_X.shape, y.shape

((399978, 370), (399978,))

In [31]:
df_X = Normalization_df(Standardzation_df(df_X))
df_X.shape

(399978, 370)

In [32]:
df_X['class'] = y.values
df_X.columns = df.columns
df_X

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,g_airmass_i_diff,...,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff,class
0,0.4003,0.4914,0.3157,0.3147,0.0080,0.7271,0.8738,0.7844,0.6806,0.5972,0.9986,0.3953,0.4146,0.1805,0.1765,0.1848,0.1826,0.1785,0.5104,0.6035,0.4778,0.9931,0.2842,0.3203,0.3799,0.3824,0.3458,0.0029,0.3757,0.3604,0.3995,0.3995,0.3994,0.3994,0.3995,0.4509,0.4568,0.9970,0.2983,0.5117,0.5239,0.4507,0.4567,0.0077,0.4295,0.3872,0.4916,0.4916,0.4917,0.4916,...,0.6091,0.6080,0.6100,0.2637,0.7363,0.6090,0.3479,0.4633,0.7195,0.0030,0.1344,0.2316,0.5256,0.6162,0.7527,0.0825,0.3294,0.3921,0.7231,0.5586,0.3731,0.3522,0.0017,0.9084,8.9604e-04,7.9167e-04,0.0211,0.7204,0.9994,6.1291e-04,5.4040e-04,0.9961,0.1764,0.1849,0.0290,0.0289,0.1805,0.0016,0.0062,0.0084,0.9777,0.0018,0.9079,0.0211,7.9334e-04,0.7182,0.9994,0.9960,5.4064e-04,0.0
1,0.2610,0.4483,0.2439,0.2219,0.0075,0.7271,0.7412,0.6584,0.5445,0.4741,0.9982,0.0000,0.0000,0.2410,0.2387,0.2433,0.2421,0.2398,0.4954,0.5765,0.4272,0.9925,0.2491,0.3196,0.3495,0.3401,0.2823,0.0025,0.4805,0.4678,0.2585,0.2585,0.2586,0.2586,0.2585,0.4459,0.4438,0.9966,0.2721,0.5256,0.5238,0.4456,0.4436,0.0074,0.4967,0.4594,0.4481,0.4480,0.4481,0.4481,...,0.4746,0.4749,0.4778,0.2637,0.7363,0.6091,0.3479,0.4633,0.7196,0.0030,0.2279,0.2948,0.3204,0.5700,0.7517,0.1008,0.3207,0.3955,0.7226,0.5577,0.3781,0.3487,0.0006,0.8581,2.5831e-04,2.3973e-04,0.0171,0.4638,0.9990,1.7514e-04,1.6126e-04,0.9933,0.2386,0.2434,0.1558,0.1557,0.2409,0.0016,0.0062,0.0084,0.9776,0.0006,0.8569,0.0171,2.4047e-04,0.4603,0.9990,0.9932,1.6085e-04,1.0
2,0.2904,0.4604,0.2708,0.2667,0.0078,0.7271,0.7679,0.6923,0.5943,0.5325,0.9985,0.0233,0.0244,0.0214,0.0223,0.0204,0.0209,0.0218,0.4951,0.5730,0.4220,0.9924,0.2565,0.3208,0.3503,0.3354,0.2765,0.0024,0.4892,0.4768,0.2920,0.2920,0.2920,0.2920,0.2920,0.4437,0.4413,0.9966,0.2795,0.5265,0.5243,0.4437,0.4413,0.0074,0.5015,0.4645,0.4618,0.4618,0.4618,0.4618,...,0.5712,0.5706,0.5730,0.2637,0.7363,0.6091,0.3479,0.4633,0.7196,0.0030,0.2181,0.1338,0.4310,0.5002,0.7612,0.0972,0.3256,0.3952,0.7225,0.5614,0.3761,0.3504,0.0008,0.8872,1.9414e-04,1.7958e-04,0.0185,0.5155,0.9992,1.2320e-04,1.1384e-04,0.9942,0.0222,0.0205,0.0534,0.0533,0.0214,0.0016,0.0062,0.0085,0.9777,0.0009,0.8890,0.0186,1.8107e-04,0.5191,0.9992,0.9943,1.1395e-04,0.0
3,0.4409,0.4972,0.3313,0.3616,0.0084,0.7271,0.9100,0.7993,0.7085,0.6578,0.9989,0.0698,0.0488,0.1954,0.1907,0.2003,0.1978,0.1931,0.5223,0.6156,0.4801,0.9931,0.2944,0.3224,0.4057,0.4024,0.3497,0.0029,0.5712,0.5729,0.4400,0.4401,0.4399,0.4400,0.4400,0.4484,0.4473,0.9967,0.3018,0.5016,0.5247,0.4487,0.4475,0.0075,0.5225,0.4939,0.4974,0.4974,0.4974,0.4974,...,0.6193,0.6182,0.6202,0.2636,0.7364,0.6094,0.3472,0.4641,0.7206,0.0029,0.2212,0.1026,0.5829,0.6151,0.7129,0.1232,0.3647,0.3805,0.7235,0.5561,0.3676,0.3485,0.0021,0.9444,8.5320e-04,7.9084e-04,0.0224,0.7905,0.9996,5.7128e-04,5.

# 훈련셋과 테스트셋 나누기

In [33]:
trn2 = df_X.iloc[:319978]
tst2 = df_X.iloc[319978:]
tst2.drop('class', axis =1 , inplace = True)
trn2.shape, tst2.shape

((319978, 371), (80000, 370))

# outlier

In [34]:
# 전체 데이터가 아닌 각 클래스별(0,1,2)로 이상치 처리
# Train_copy1 = Train_set.copy()

def remove_outlier(data, column):
    
    print(column)
    # 클래스 0처리
    column_data_0 = data[data['class']==0][column]
    quan_25_0 = np.percentile(column_data_0.values, 0.1) # 1분위수
    quan_75_0 = np.percentile(column_data_0.values, 99.9) # 3분위수
    iqr_0 = quan_75_0 - quan_25_0 # IQR 계산
    iqr_0 *= 1.5
    lowest_0 = quan_25_0 - iqr_0 # 최솟값 설정
    highest_0 = quan_75_0 + iqr_0 # 최댓값 설정
    outlier_index_0 = column_data_0[(column_data_0 < lowest_0) | (column_data_0 > highest_0)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_0))
    
    # 클래스 1처리
    column_data_1 = data[data['class']==1][column]
    quan_25_1 = np.percentile(column_data_1.values, 0.1) # 1분위수
    quan_75_1 = np.percentile(column_data_1.values, 99.9) # 3분위수
    iqr_1 = quan_75_1 - quan_25_1 # IQR 계산
    iqr_1 *= 1.5
    lowest_1 = quan_25_1 - iqr_1 # 최솟값 설정
    highest_1 = quan_75_1 + iqr_1 # 최댓값 설정
    outlier_index_1 = column_data_1[(column_data_1 < lowest_1 )| (column_data_1 > highest_1)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_1))
    
    # 클래스 2 처리
    column_data_2 = data[data['class']==2][column]
    quan_25_2 = np.percentile(column_data_2.values, 0.1) # 1분위수
    quan_75_2 = np.percentile(column_data_2.values, 99.9) # 3분위수
    iqr_2 = quan_75_2 - quan_25_2 # IQR 계산
    iqr_2 *= 1.5
    lowest_2 = quan_25_2 - iqr_2 # 최솟값 설정
    highest_2 = quan_75_2 + iqr_2 # 최댓값 설정
    outlier_index_2 = column_data_2[(column_data_2 < lowest_2 )| (column_data_2 > highest_2)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_2))
    # outlier 행 제거
    data.drop(list(set(list(outlier_index_0) + list(outlier_index_1) + list(outlier_index_2))), axis = 0, inplace = True)
    print(data.shape)
    return data

In [35]:
col = list(trn2.columns)
col.remove('redshift')
col.remove('class')
# col

In [36]:
df2 = trn2.copy()
for i in col:
    
    ROT = remove_outlier(df2, i)
    
    df2 = ROT

df2

u
5
1
2
(319970, 371)
g
0
0
0
(319970, 371)
r
1
0
2
(319967, 371)
i
1
1
0
(319965, 371)
z
1
1
0
(319963, 371)
dered_u
0
0
0
(319963, 371)
dered_g
0
0
0
(319963, 371)
dered_r
0
0
0
(319963, 371)
dered_i
0
0
0
(319963, 371)
dered_z
0
0
0
(319963, 371)
nObserve
0
0
0
(319963, 371)
nDetect
0
0
0
(319963, 371)
airmass_u
0
0
0
(319963, 371)
airmass_g
0
0
0
(319963, 371)
airmass_r
0
0
0
(319963, 371)
airmass_i
0
0
0
(319963, 371)
airmass_z
0
0
0
(319963, 371)
u_g_diff
1
4
7
(319951, 371)
u_r_diff
0
1
4
(319946, 371)
u_i_diff
0
0
1
(319945, 371)
u_z_diff
0
1
1
(319943, 371)
u_redshift_diff
24
0
0
(319919, 371)
u_dered_u_diff
1
8
18
(319892, 371)
u_dered_g_diff
0
0
0
(319892, 371)
u_dered_r_diff
0
0
0
(319892, 371)
u_dered_i_diff
0
0
0
(319892, 371)
u_dered_z_diff
0
0
0
(319892, 371)
u_nObserve_diff
0
0
0
(319892, 371)
u_nDetect_diff
0
0
0
(319892, 371)
u_airmass_u_diff
0
0
0
(319892, 371)
u_airmass_g_diff
0
0
0
(319892, 371)
u_airmass_r_diff
0
0
0
(319892, 371)
u_airmass_i_diff
0
0
0
(319892, 

0
0
(319150, 371)
i_z_divide
1
0
3
(319146, 371)
i_redshift_divide
1
0
0
(319145, 371)
i_dered_u_divide
0
0
0
(319145, 371)
i_dered_g_divide
0
0
0
(319145, 371)
i_dered_r_divide
0
4
0
(319141, 371)
i_dered_i_divide
0
0
0
(319141, 371)
i_dered_z_divide
0
0
0
(319141, 371)
i_nObserve_divide
0
0
0
(319141, 371)
i_nDetect_divide
0
0
0
(319141, 371)
i_airmass_u_divide
0
0
0
(319141, 371)
i_airmass_g_divide
0
0
0
(319141, 371)
i_airmass_r_divide
0
0
0
(319141, 371)
i_airmass_i_divide
0
0
0
(319141, 371)
i_airmass_z_divide
0
0
0
(319141, 371)
z_redshift_divide
0
1
1
(319139, 371)
z_dered_u_divide
0
0
0
(319139, 371)
z_dered_g_divide
2
0
0
(319137, 371)
z_dered_r_divide
0
4
0
(319133, 371)
z_dered_i_divide
0
4
5
(319124, 371)
z_dered_z_divide
0
1
0
(319123, 371)
z_nObserve_divide
0
0
0
(319123, 371)
z_nDetect_divide
0
0
0
(319123, 371)
z_airmass_u_divide
0
0
0
(319123, 371)
z_airmass_g_divide
0
0
0
(319123, 371)
z_airmass_r_divide
0
0
0
(319123, 371)
z_airmass_i_divide
0
0
0
(319123, 371)
z_ai

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,g_airmass_i_diff,...,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff,class
0,0.4003,0.4914,0.3157,0.3147,0.0080,0.7271,0.8738,0.7844,0.6806,0.5972,0.9986,0.3953,0.4146,0.1805,0.1765,0.1848,0.1826,0.1785,0.5104,0.6035,0.4778,0.9931,0.2842,0.3203,0.3799,0.3824,0.3458,0.0029,0.3757,0.3604,0.3995,0.3995,0.3994,0.3994,0.3995,0.4509,0.4568,0.9970,0.2983,0.5117,0.5239,0.4507,0.4567,0.0077,0.4295,0.3872,0.4916,0.4916,0.4917,0.4916,...,0.6091,0.6080,0.6100,0.2637,0.7363,0.6090,0.3479,0.4633,0.7195,0.0030,0.1344,0.2316,0.5256,0.6162,0.7527,0.0825,0.3294,0.3921,0.7231,0.5586,0.3731,0.3522,0.0017,0.9084,0.0009,0.0008,0.0211,0.7204,0.9994,0.0006,0.0005,0.9961,0.1764,0.1849,0.0290,0.0289,0.1805,0.0016,0.0062,0.0084,0.9777,0.0018,0.9079,0.0211,0.0008,0.7182,0.9994,0.9960,0.0005,0.0
1,0.2610,0.4483,0.2439,0.2219,0.0075,0.7271,0.7412,0.6584,0.5445,0.4741,0.9982,0.0000,0.0000,0.2410,0.2387,0.2433,0.2421,0.2398,0.4954,0.5765,0.4272,0.9925,0.2491,0.3196,0.3495,0.3401,0.2823,0.0025,0.4805,0.4678,0.2585,0.2585,0.2586,0.2586,0.2585,0.4459,0.4438,0.9966,0.2721,0.5256,0.5238,0.4456,0.4436,0.0074,0.4967,0.4594,0.4481,0.4480,0.4481,0.4481,...,0.4746,0.4749,0.4778,0.2637,0.7363,0.6091,0.3479,0.4633,0.7196,0.0030,0.2279,0.2948,0.3204,0.5700,0.7517,0.1008,0.3207,0.3955,0.7226,0.5577,0.3781,0.3487,0.0006,0.8581,0.0003,0.0002,0.0171,0.4638,0.9990,0.0002,0.0002,0.9933,0.2386,0.2434,0.1558,0.1557,0.2409,0.0016,0.0062,0.0084,0.9776,0.0006,0.8569,0.0171,0.0002,0.4603,0.9990,0.9932,0.0002,1.0
2,0.2904,0.4604,0.2708,0.2667,0.0078,0.7271,0.7679,0.6923,0.5943,0.5325,0.9985,0.0233,0.0244,0.0214,0.0223,0.0204,0.0209,0.0218,0.4951,0.5730,0.4220,0.9924,0.2565,0.3208,0.3503,0.3354,0.2765,0.0024,0.4892,0.4768,0.2920,0.2920,0.2920,0.2920,0.2920,0.4437,0.4413,0.9966,0.2795,0.5265,0.5243,0.4437,0.4413,0.0074,0.5015,0.4645,0.4618,0.4618,0.4618,0.4618,...,0.5712,0.5706,0.5730,0.2637,0.7363,0.6091,0.3479,0.4633,0.7196,0.0030,0.2181,0.1338,0.4310,0.5002,0.7612,0.0972,0.3256,0.3952,0.7225,0.5614,0.3761,0.3504,0.0008,0.8872,0.0002,0.0002,0.0185,0.5155,0.9992,0.0001,0.0001,0.9942,0.0222,0.0205,0.0534,0.0533,0.0214,0.0016,0.0062,0.0085,0.9777,0.0009,0.8890,0.0186,0.0002,0.5191,0.9992,0.9943,0.0001,0.0
3,0.4409,0.4972,0.3313,0.3616,0.0084,0.7271,0.9100,0.7993,0.7085,0.6578,0.9989,0.0698,0.0488,0.1954,0.1907,0.2003,0.1978,0.1931,0.5223,0.6156,0.4801,0.9931,0.2944,0.3224,0.4057,0.4024,0.3497,0.0029,0.5712,0.5729,0.4400,0.4401,0.4399,0.4400,0.4400,0.4484,0.4473,0.9967,0.3018,0.5016,0.5247,0.4487,0.4475,0.0075,0.5225,0.4939,0.4974,0.4974,0.4974,0.4974,...,0.6193,0.6182,0.6202,0.2636,0.7364,0.6094,0.3472,0.4641,0.7206,0.0029,0.2212,0.1026,0.5829,0.6151,0.7129,0.1232,0.3647,0.3805,0.7235,0.5561,0.3676,0.3485,0.0021,0.9444,0.0009,0.0008,0.0224,0.7905,0.9996,0.0006,0.0005,0.9969,0.1906,0.2004,0.0200,0.0200,0.1954,0.0016,0.0062,0.0085,0.9777,0.0021,0.

In [37]:
target_col = 'class'
feature_col = [x for x in df.columns if df[x].dtype in [np.int64, np.float64] and x !=target_col]
print(feature_col)
pred_col='pred'


['u', 'g', 'r', 'i', 'z', 'redshift', 'dered_u', 'dered_g', 'dered_r', 'dered_i', 'dered_z', 'nObserve', 'nDetect', 'airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z', 'u_g_diff', 'u_r_diff', 'u_i_diff', 'u_z_diff', 'u_redshift_diff', 'u_dered_u_diff', 'u_dered_g_diff', 'u_dered_r_diff', 'u_dered_i_diff', 'u_dered_z_diff', 'u_nObserve_diff', 'u_nDetect_diff', 'u_airmass_u_diff', 'u_airmass_g_diff', 'u_airmass_r_diff', 'u_airmass_i_diff', 'u_airmass_z_diff', 'g_r_diff', 'g_i_diff', 'g_z_diff', 'g_redshift_diff', 'g_dered_u_diff', 'g_dered_g_diff', 'g_dered_r_diff', 'g_dered_i_diff', 'g_dered_z_diff', 'g_nObserve_diff', 'g_nDetect_diff', 'g_airmass_u_diff', 'g_airmass_g_diff', 'g_airmass_r_diff', 'g_airmass_i_diff', 'g_airmass_z_diff', 'r_i_diff', 'r_z_diff', 'r_redshift_diff', 'r_dered_u_diff', 'r_dered_g_diff', 'r_dered_r_diff', 'r_dered_i_diff', 'r_dered_z_diff', 'r_nObserve_diff', 'r_nDetect_diff', 'r_airmass_u_diff', 'r_airmass_g_diff', 'r_airmass_r_diff', 'r_airmass_i_

In [38]:
from sklearn.feature_selection import SelectFromModel
#LGBM 모델을 기반으로 속성들 선택
select = SelectFromModel(
     lgb.LGBMClassifier(objective='multiclass',
                         n_estimators=20000,
                         num_leaves= 512,
                         learning_rate=0.001,
                         min_child_samples=10,
                         subsample=.5,
                         subsample_freq=1,
                         colsample_bytree=.8,
                         random_state=seed,
                         max_depth = 27,
                         #feature_fraction = 0.7,
                         #bagging_fraction = 0.7,
                         n_jobs=-1,
                        boosting_type='dart')
    )#, threshold = 0.3)
select.fit(df2[feature_col], df2[target_col])

train_sel = select.transform(df2[feature_col])

test_sel = select.transform(tst2[feature_col])

## 모델링

In [40]:
df2.shape

(318740, 371)

In [41]:
train_sel.shape , type(train_sel)

((318740, 124), numpy.ndarray)

In [42]:
train_sel

array([[0.40030086, 0.49137559, 0.31567054, ..., 0.00622441, 0.00844638,
        0.977652  ],
       [0.26102761, 0.44825998, 0.2439184 , ..., 0.00622477, 0.00844093,
        0.9776369 ],
       [0.29044048, 0.46037709, 0.27082537, ..., 0.00622672, 0.00845781,
        0.97765483],
       ...,
       [0.36510889, 0.47822807, 0.29354107, ..., 0.00622619, 0.00845582,
        0.9776645 ],
       [0.32539193, 0.47063707, 0.28334225, ..., 0.00629406, 0.00885908,
        0.97809486],
       [0.3425768 , 0.48095257, 0.30929311, ..., 0.00622119, 0.00843283,
        0.97762861]])

# Stratified K-Fold Cross Validation

In [56]:
train_df = pd.DataFrame(train_sel)
train_df['class'] = y.iloc[list(df2.index)].values
train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,class
0,0.4003,0.4914,0.3157,0.0080,0.7271,0.1805,0.5104,0.6035,0.4778,0.3203,0.3799,0.3824,0.3757,0.4509,0.4568,0.9970,0.5117,0.5239,0.4507,0.4567,0.0077,0.3870,0.9929,0.2833,0.3456,0.3921,0.3294,0.0030,0.9928,0.1786,0.2808,0.3158,0.4589,0.0018,0.0077,0.0087,0.0070,0.0072,0.0037,0.7158,0.7270,0.7270,0.6303,0.6788,0.7378,0.5355,0.5867,0.0022,0.5107,0.0020,...,0.0836,0.4042,0.9138,0.0125,0.0055,0.0060,0.0362,0.0095,0.0352,0.6658,0.6750,0.6180,0.6180,0.7225,0.7222,0.5558,0.0701,0.4608,0.9940,0.0777,0.4838,0.9877,0.5136,0.9761,0.9452,0.6090,0.3479,0.4633,0.7195,0.0030,0.2316,0.5256,0.6162,0.7527,0.0825,0.3294,0.3921,0.7231,0.5586,0.3731,0.3522,0.0008,0.0006,0.0005,0.0290,0.0289,0.0062,0.0084,0.9777,0.0
1,0.2610,0.4483,0.2439,0.0075,0.7271,0.2410,0.4954,0.5765,0.4272,0.3196,0.3495,0.3401,0.4805,0.4459,0.4438,0.9966,0.5256,0.5238,0.4456,0.4436,0.0074,0.3741,0.9927,0.3147,0.3540,0.3919,0.3153,0.0028,0.9927,0.2355,0.3146,0.3382,0.4587,0.0018,0.0084,0.0091,0.0073,0.0073,0.0037,0.7506,0.7268,0.7267,0.5614,0.6107,0.6522,0.5120,0.5248,0.0020,0.4713,0.0018,...,0.0885,0.4133,0.9162,0.0121,0.0058,0.0062,0.0363,0.0095,0.0352,0.6658,0.6750,0.6181,0.6181,0.7226,0.7222,0.5107,0.0635,0.4469,0.9924,0.0747,0.4717,0.9862,0.4904,0.9738,0.9434,0.6091,0.3479,0.4633,0.7196,0.0030,0.2948,0.3204,0.5700,0.7517,0.1008,0.3207,0.3955,0.7226,0.5577,0.3781,0.3487,0.0002,0.0002,0.0002,0.1558,0.1557,0.0062,0.0084,0.9776,1.0
2,0.2904,0.4604,0.2708,0.0078,0.7271,0.0214,0.4951,0.5730,0.4220,0.3208,0.3503,0.3354,0.4892,0.4437,0.4413,0.9966,0.5265,0.5243,0.4437,0.4413,0.0074,0.3737,0.9927,0.3200,0.3588,0.3926,0.3153,0.0028,0.9927,0.2428,0.3227,0.3401,0.4595,0.0018,0.0084,0.0091,0.0073,0.0073,0.0037,0.7436,0.7277,0.7277,0.5599,0.6006,0.6421,0.5005,0.5113,0.0019,0.4693,0.0018,...,0.0901,0.4174,0.9165,0.0120,0.0059,0.0062,0.0364,0.0095,0.0352,0.6658,0.6750,0.6180,0.6180,0.7225,0.7222,0.5045,0.0610,0.4433,0.9921,0.0713,0.4672,0.9858,0.4883,0.9735,0.9431,0.6091,0.3479,0.4633,0.7196,0.0030,0.1338,0.4310,0.5002,0.7612,0.0972,0.3256,0.3952,0.7225,0.5614,0.3761,0.3504,0.0002,0.0001,0.0001,0.0534,0.0533,0.0062,0.0085,0.9777,0.0
3,0.4409,0.4972,0.3313,0.0084,0.7271,0.1954,0.5223,0.6156,0.4801,0.3224,0.4057,0.4024,0.5712,0.4484,0.4473,0.9967,0.5016,0.5247,0.4487,0.4475,0.0075,0.3757,0.9927,0.2710,0.3513,0.3931,0.3178,0.0028,0.9928,0.1784,0.3079,0.3372,0.4601,0.0018,0.0078,0.0090,0.0073,0.0073,0.0037,0.7063,0.7269,0.7269,0.6843,0.7074,0.7393,0.5224,0.5391,0.0020,0.4753,0.0018,...,0.0844,0.4133,0.9162,0.0121,0.0056,0.0061,0.0363,0.0095,0.0352,0.6658,0.6750,0.6180,0.6180,0.7225,0.7222,0.5968,0.0727,0.4585,0.9935,0.0749,0.4721,0.9863,0.4910,0.9739,0.9436,0.6094,0.3472,0.4641,0.7206,0.0029,0.1026,0.5829,0.6151,0.7129,0.1232,0.3647,0.3805,0.7235,0.5561,0.3676,0.3485,0.0008,0.0006,0.0005,0.0200,0.0200,0.0062,0.0085,0.9777,0.0
4,0.4205,0.4939,0.3161,0.0082,0.7271,0.1845,0.5168,0.6153,0.4849,0.3271,0.3957,0.4027,0.4495,0.4532,0.4548,0.9969,0.5096,0.5252,0.4539,0.4553,0.0076,0.3801,0.9928,0.2759,0.3439,0.3937,0.3230,0.0029,0.9928,0.1786,0.2895,0.3304,0.4608,0.0018,0.0078,0.0088,0.0072,0.0072,0.0037,0.7130,0.7270,0.7270,0.6474,0.6981,0.7398,0.5444,0.5739,0.0021,0.4884,0.0019,...,0.0839,0.4071,0.9155,0.0122,0.0055,0.0060,0.0363,0.0095,0.0352,0.6658,0.6750,0.6180,0.6180,0.7225,0.7222,0.5693,0.0726,0.4605,0.9938,0.0795,0.4804,0.9871,0.4993,0.9746,0.9439,0.6090,0.3480,0.4634,0.7195,0.0030,0.1365,0.5459,0.6197,0.7418,0.1102,0.3498,0.3891,0.7234,0.5709,0.3749,0.3470,0.0008,0.0006,0.0005,0.0091,0.0091,0.0062,0.0085,0.9778,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [60]:
train_df.to_csv('/home/swcu/Desktop/jjj_trn.csv')

In [57]:
test_df = pd.DataFrame(test_sel)
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123
0,0.3304,0.4754,0.2958,0.0080,0.7276,0.1487,0.4964,0.5772,0.4304,0.3194,0.3517,0.3412,0.5293,0.4454,0.4445,0.9967,0.5245,0.5238,0.4451,0.4443,0.0074,0.3761,0.9927,0.3136,0.3548,0.3919,0.3174,0.0028,0.9927,0.2315,0.3127,0.3348,0.4586,0.0018,0.0083,0.0090,0.0073,0.0073,0.0037,0.7336,0.7277,0.7277,0.5671,0.6130,0.6581,0.5097,0.5282,0.0020,0.4772,0.0018,...,0.9099,0.0889,0.4140,0.9159,0.0121,0.0058,0.0062,0.0363,0.0095,0.0352,0.6664,0.6756,0.6192,0.6192,0.7231,0.7228,0.5068,0.0622,0.4458,0.9924,0.0730,0.4708,0.9862,0.4930,0.9740,0.9435,0.6091,0.3479,0.4633,0.7196,0.0029,0.4284,0.4559,0.5424,0.7234,0.0924,0.3267,0.3942,0.7235,0.5632,0.3760,0.3494,2.7489e-04,2.0855e-04,1.8633e-04,0.1688,0.1688,0.0062,0.0084,0.9776
1,0.3236,0.4696,0.2816,0.0079,0.7271,0.0072,0.4998,0.5844,0.4401,0.3642,0.4021,0.3830,0.5125,0.4472,0.4465,0.9967,0.5477,0.5430,0.4605,0.4563,0.0076,0.3763,0.9927,0.3486,0.3852,0.4170,0.3352,0.0030,0.9928,0.2751,0.3587,0.3723,0.4888,0.0019,0.0087,0.0094,0.0075,0.0074,0.0037,0.7501,0.7278,0.7278,0.5541,0.5879,0.6254,0.4896,0.4923,0.0018,0.4610,0.0017,...,0.9101,0.0955,0.4338,0.9212,0.0130,0.0061,0.0065,0.0365,0.0097,0.0361,0.6658,0.6750,0.6180,0.6180,0.7225,0.7222,0.5010,0.0592,0.4397,0.9917,0.0687,0.4622,0.9852,0.4821,0.9729,0.9425,0.6090,0.3479,0.4633,0.7196,0.0030,0.1357,0.4351,0.5043,0.7708,0.1105,0.3558,0.4320,0.7227,0.5651,0.3731,0.3475,3.7888e-04,6.8173e-05,5.8546e-05,0.0099,0.0098,0.0064,0.0091,0.9783
2,0.3330,0.4745,0.2926,0.0080,0.7276,0.3624,0.4987,0.5814,0.4365,0.3210,0.3569,0.3482,0.5311,0.4462,0.4457,0.9967,0.5233,0.5241,0.4462,0.4457,0.0075,0.3767,0.9927,0.3103,0.3540,0.3924,0.3184,0.0028,0.9927,0.2265,0.3104,0.3344,0.4593,0.0018,0.0083,0.0090,0.0073,0.0073,0.0037,0.7334,0.7267,0.7266,0.5746,0.6211,0.6663,0.5131,0.5332,0.0020,0.4789,0.0018,...,0.9099,0.0882,0.4131,0.9158,0.0121,0.0058,0.0061,0.0363,0.0095,0.0352,0.6663,0.6755,0.6191,0.6191,0.7229,0.7226,0.5144,0.0634,0.4476,0.9925,0.0738,0.4721,0.9863,0.4943,0.9741,0.9435,0.6091,0.3479,0.4633,0.7196,0.0029,0.4106,0.4122,0.5953,0.6972,0.0933,0.3296,0.3950,0.7237,0.5631,0.3760,0.3495,3.3547e-04,2.4502e-04,2.2030e-04,0.3263,0.3263,0.0062,0.0085,0.9777
3,0.3123,0.4695,0.2885,0.0080,0.7271,0.1339,0.4948,0.5722,0.4203,0.3199,0.3489,0.3336,0.5166,0.4434,0.4406,0.9966,0.5262,0.5239,0.4432,0.4405,0.0074,0.3731,0.9927,0.3200,0.3586,0.3921,0.3143,0.0028,0.9927,0.2436,0.3233,0.3403,0.4589,0.0017,0.0085,0.0091,0.0073,0.0073,0.0037,0.7377,0.7272,0.7272,0.5591,0.5995,0.6402,0.4998,0.5093,0.0019,0.4679,0.0018,...,0.9095,0.0904,0.4178,0.9165,0.0120,0.0059,0.0062,0.0364,0.0095,0.0352,0.6658,0.6750,0.6180,0.6180,0.7225,0.7222,0.5007,0.0605,0.4423,0.9920,0.0710,0.4664,0.9857,0.4871,0.9734,0.9430,0.6090,0.3479,0.4634,0.7196,0.0029,0.4082,0.4376,0.5189,0.7219,0.0931,0.3289,0.3942,0.7229,0.5630,0.3758,0.3498,1.6201e-04,1.1267e-04,1.0604e-04,0.0017,0.0017,0.0062,0.0084,0.9776
4,0.3082,0.4668,0.2796,0.0079,0.7274,0.1056,0.4960,0.5767,0.4295,0.3195,0.3512,0.3407,0.5017,0.4455,0.4445,0.9967,0.5250,0.5240,0.4453,0.4443,0.0074,0.3759,0.9927,0.3143,0.3550,0.3921,0.3173,0.0028,0.9927,0.2327,0.3133,0.3354,0.4589,0.0018,0.0083,0.0090,0.0073,0.0072,0.0037,0.7390,0.7277,0.7277,0.5655,0.6118,0.6566,0.5097,0.5274,0.0020,0.4764,0.0018,...,0.9100,0.0888,0.4138,0.9159,0.0121,0.0058,0.0062,0.0363,0.0095,0.0352,0.6661,0.6754,0.6183,0.6183,0.7229,0.7225,0.5080,0.0625,0.4462,0.9924,0.0733,0.4711,0.9862,0.4931,0.9740,0.9436,0.6091,0.3479,0.4633,0.7196,0.0030,0.1405,0.4373,0.5361,0.7455,0.0955,0.3235,0.3950,0.7230,0.5616,0.3764,0.3495,2.6801e-04,2.0304e-04,1.8092e-04,0.1176,0.1175,0.0062,0.0084,0.9776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [58]:
y = train_df['class'].values
trn = train_df.drop('class', axis = 1).values
tst = test_df.values
trn.shape, y.shape, tst.shape

((318740, 124), (318740,), (80000, 124))

In [ ]:
n_class =3
n_fold = 10
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
   # print(i, i_trn, i_val)
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             boosting_type='dart',
                             n_estimators=20000,
                             num_leaves=512,
                             learning_rate=0.001,
                             min_child_samples=10,
                             subsample=.5,
                             subsample_freq=1,
                             colsample_bytree=.8,
                             random_state=seed,
                             #feature_fraction = 0.9,
                             #bagging_fraction = 0.7,
                             n_jobs=-1)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=100)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

# np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
# np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

training model for CV #1
[1]	valid_0's multi_logloss: 0.984418
[2]	valid_0's multi_logloss: 0.982847
[3]	valid_0's multi_logloss: 0.981284
[4]	valid_0's multi_logloss: 0.979722
[5]	valid_0's multi_logloss: 0.978167
[6]	valid_0's multi_logloss: 0.976618
[7]	valid_0's multi_logloss: 0.976624
[8]	valid_0's multi_logloss: 0.975076
[9]	valid_0's multi_logloss: 0.973535
[10]	valid_0's multi_logloss: 0.973544
[11]	valid_0's multi_logloss: 0.972008
[12]	valid_0's multi_logloss: 0.97227
[13]	valid_0's multi_logloss: 0.970735
[14]	valid_0's multi_logloss: 0.969208
[15]	valid_0's multi_logloss: 0.967682
[16]	valid_0's multi_logloss: 0.966158
[17]	valid_0's multi_logloss: 0.966671
[18]	valid_0's multi_logloss: 0.965154
[19]	valid_0's multi_logloss: 0.965157
[20]	valid_0's multi_logloss: 0.965244
[21]	valid_0's multi_logloss: 0.96576
[22]	valid_0's multi_logloss: 0.970321
[23]	valid_0's multi_logloss: 0.968842
[24]	valid_0's multi_logloss: 0.978333
[25]	valid_0's multi_logloss: 0.978013
[26]	valid_

[209]	valid_0's multi_logloss: 0.969075
[210]	valid_0's multi_logloss: 0.967678
[211]	valid_0's multi_logloss: 0.966285
[212]	valid_0's multi_logloss: 0.964891
[213]	valid_0's multi_logloss: 0.963499
[214]	valid_0's multi_logloss: 0.966
[215]	valid_0's multi_logloss: 0.964611
[216]	valid_0's multi_logloss: 0.963222
[217]	valid_0's multi_logloss: 0.963761
[218]	valid_0's multi_logloss: 0.964327
[219]	valid_0's multi_logloss: 0.962941
[220]	valid_0's multi_logloss: 0.963549
[221]	valid_0's multi_logloss: 0.964204
[222]	valid_0's multi_logloss: 0.962817
[223]	valid_0's multi_logloss: 0.961435
[224]	valid_0's multi_logloss: 0.960054
[225]	valid_0's multi_logloss: 0.958679
[226]	valid_0's multi_logloss: 0.957305
[227]	valid_0's multi_logloss: 0.957949
[228]	valid_0's multi_logloss: 0.956578
[229]	valid_0's multi_logloss: 0.955206
[230]	valid_0's multi_logloss: 0.95384
[231]	valid_0's multi_logloss: 0.954613
[232]	valid_0's multi_logloss: 0.953248
[233]	valid_0's multi_logloss: 0.951887
[234

[415]	valid_0's multi_logloss: 0.91669
[416]	valid_0's multi_logloss: 0.915411
[417]	valid_0's multi_logloss: 0.914129
[418]	valid_0's multi_logloss: 0.912854
[419]	valid_0's multi_logloss: 0.91158
[420]	valid_0's multi_logloss: 0.912239
[421]	valid_0's multi_logloss: 0.91288
[422]	valid_0's multi_logloss: 0.911605
[423]	valid_0's multi_logloss: 0.912236
[424]	valid_0's multi_logloss: 0.912889
[425]	valid_0's multi_logloss: 0.913444
[426]	valid_0's multi_logloss: 0.914038
[427]	valid_0's multi_logloss: 0.912761
[428]	valid_0's multi_logloss: 0.914144
[429]	valid_0's multi_logloss: 0.914799
[430]	valid_0's multi_logloss: 0.915446
[431]	valid_0's multi_logloss: 0.914169
[432]	valid_0's multi_logloss: 0.914785
[433]	valid_0's multi_logloss: 0.913508
[434]	valid_0's multi_logloss: 0.914033
[435]	valid_0's multi_logloss: 0.91276
[436]	valid_0's multi_logloss: 0.911484
[437]	valid_0's multi_logloss: 0.91217
[438]	valid_0's multi_logloss: 0.9109
[439]	valid_0's multi_logloss: 0.911516
[440]	v

[621]	valid_0's multi_logloss: 0.859438
[622]	valid_0's multi_logloss: 0.860593
[623]	valid_0's multi_logloss: 0.861157
[624]	valid_0's multi_logloss: 0.859993
[625]	valid_0's multi_logloss: 0.858833
[626]	valid_0's multi_logloss: 0.857672
[627]	valid_0's multi_logloss: 0.858064
[628]	valid_0's multi_logloss: 0.858643
[629]	valid_0's multi_logloss: 0.859731
[630]	valid_0's multi_logloss: 0.858569
[631]	valid_0's multi_logloss: 0.859158
[632]	valid_0's multi_logloss: 0.857997
[633]	valid_0's multi_logloss: 0.858573
[634]	valid_0's multi_logloss: 0.857414
[635]	valid_0's multi_logloss: 0.858482
[636]	valid_0's multi_logloss: 0.857325
[637]	valid_0's multi_logloss: 0.856169
[638]	valid_0's multi_logloss: 0.855011
[639]	valid_0's multi_logloss: 0.853859
[640]	valid_0's multi_logloss: 0.85271
[641]	valid_0's multi_logloss: 0.853313
[642]	valid_0's multi_logloss: 0.854375
[643]	valid_0's multi_logloss: 0.853226
[644]	valid_0's multi_logloss: 0.852077
[645]	valid_0's multi_logloss: 0.852542
[

[827]	valid_0's multi_logloss: 0.77803
[828]	valid_0's multi_logloss: 0.778529
[829]	valid_0's multi_logloss: 0.77753
[830]	valid_0's multi_logloss: 0.778062
[831]	valid_0's multi_logloss: 0.777063
[832]	valid_0's multi_logloss: 0.777477
[833]	valid_0's multi_logloss: 0.776479
[834]	valid_0's multi_logloss: 0.775482
[835]	valid_0's multi_logloss: 0.776056
[836]	valid_0's multi_logloss: 0.776456
[837]	valid_0's multi_logloss: 0.775459
[838]	valid_0's multi_logloss: 0.776412
[839]	valid_0's multi_logloss: 0.775415
[840]	valid_0's multi_logloss: 0.774423
[841]	valid_0's multi_logloss: 0.773428
[842]	valid_0's multi_logloss: 0.772439
[843]	valid_0's multi_logloss: 0.772858
[844]	valid_0's multi_logloss: 0.771868
[845]	valid_0's multi_logloss: 0.770883
[846]	valid_0's multi_logloss: 0.771378
[847]	valid_0's multi_logloss: 0.77185
[848]	valid_0's multi_logloss: 0.770864
[849]	valid_0's multi_logloss: 0.771495
[850]	valid_0's multi_logloss: 0.771967
[851]	valid_0's multi_logloss: 0.770979
[85

...

[842]	valid_0's multi_logloss: 0.771938
[843]	valid_0's multi_logloss: 0.772358
[844]	valid_0's multi_logloss: 0.771365
[845]	valid_0's multi_logloss: 0.770375
[846]	valid_0's multi_logloss: 0.770871
[847]	valid_0's multi_logloss: 0.771344
[848]	valid_0's multi_logloss: 0.770356
[849]	valid_0's multi_logloss: 0.770988
[850]	valid_0's multi_logloss: 0.771461
[851]	valid_0's multi_logloss: 0.770473
[852]	valid_0's multi_logloss: 0.770863
[853]	valid_0's multi_logloss: 0.769874
[854]	valid_0's multi_logloss: 0.770328
[855]	valid_0's multi_logloss: 0.771241
[856]	valid_0's multi_logloss: 0.770251
[857]	valid_0's multi_logloss: 0.769264
[858]	valid_0's multi_logloss: 0.769782
[859]	valid_0's multi_logloss: 0.770313
[860]	valid_0's multi_logloss: 0.769327
[861]	valid_0's multi_logloss: 0.769884
[862]	valid_0's multi_logloss: 0.768899
[863]	valid_0's multi_logloss: 0.769336
[864]	valid_0's multi_logloss: 0.769692
[865]	valid_0's multi_logloss: 0.768705
[866]	valid_0's multi_logloss: 0.769076


[1047]	valid_0's multi_logloss: 0.727887
[1048]	valid_0's multi_logloss: 0.72828
[1049]	valid_0's multi_logloss: 0.727375
[1050]	valid_0's multi_logloss: 0.72647
[1051]	valid_0's multi_logloss: 0.725565
[1052]	valid_0's multi_logloss: 0.725938
[1053]	valid_0's multi_logloss: 0.725035
[1054]	valid_0's multi_logloss: 0.725343
[1055]	valid_0's multi_logloss: 0.725764
[1056]	valid_0's multi_logloss: 0.724863
[1057]	valid_0's multi_logloss: 0.725366
[1058]	valid_0's multi_logloss: 0.724463
[1059]	valid_0's multi_logloss: 0.723565
[1060]	valid_0's multi_logloss: 0.722668
[1061]	valid_0's multi_logloss: 0.722964
[1062]	valid_0's multi_logloss: 0.722068
[1063]	valid_0's multi_logloss: 0.721172
[1064]	valid_0's multi_logloss: 0.720279
[1065]	valid_0's multi_logloss: 0.720742
[1066]	valid_0's multi_logloss: 0.721214
[1067]	valid_0's multi_logloss: 0.721657
[1068]	valid_0's multi_logloss: 0.720762
[1069]	valid_0's multi_logloss: 0.719869
[1070]	valid_0's multi_logloss: 0.718977
[1071]	valid_0's m

[1248]	valid_0's multi_logloss: 0.676668
[1249]	valid_0's multi_logloss: 0.675856
[1250]	valid_0's multi_logloss: 0.676183
[1251]	valid_0's multi_logloss: 0.676796
[1252]	valid_0's multi_logloss: 0.677028
[1253]	valid_0's multi_logloss: 0.677297
[1254]	valid_0's multi_logloss: 0.676485
[1255]	valid_0's multi_logloss: 0.675672
[1256]	valid_0's multi_logloss: 0.676404
[1257]	valid_0's multi_logloss: 0.676811
[1258]	valid_0's multi_logloss: 0.677158
[1259]	valid_0's multi_logloss: 0.676348
[1260]	valid_0's multi_logloss: 0.676697
[1261]	valid_0's multi_logloss: 0.677044
[1262]	valid_0's multi_logloss: 0.677378
[1263]	valid_0's multi_logloss: 0.676565
[1264]	valid_0's multi_logloss: 0.676909
[1265]	valid_0's multi_logloss: 0.677301
[1266]	valid_0's multi_logloss: 0.67649
[1267]	valid_0's multi_logloss: 0.676821
[1268]	valid_0's multi_logloss: 0.677192
[1269]	valid_0's multi_logloss: 0.677575
[1270]	valid_0's multi_logloss: 0.67796
[1271]	valid_0's multi_logloss: 0.677146
[1272]	valid_0's m

[1449]	valid_0's multi_logloss: 0.636794
[1450]	valid_0's multi_logloss: 0.637055
[1451]	valid_0's multi_logloss: 0.637387
[1452]	valid_0's multi_logloss: 0.636645
[1453]	valid_0's multi_logloss: 0.636995
[1454]	valid_0's multi_logloss: 0.636256
[1455]	valid_0's multi_logloss: 0.635516
[1456]	valid_0's multi_logloss: 0.635853
[1457]	valid_0's multi_logloss: 0.636153
[1458]	valid_0's multi_logloss: 0.636416
[1459]	valid_0's multi_logloss: 0.636886
[1460]	valid_0's multi_logloss: 0.637258
[1461]	valid_0's multi_logloss: 0.637581
[1462]	valid_0's multi_logloss: 0.637867
[1463]	valid_0's multi_logloss: 0.637121
[1464]	valid_0's multi_logloss: 0.637405
[1465]	valid_0's multi_logloss: 0.637692
[1466]	valid_0's multi_logloss: 0.637963
[1467]	valid_0's multi_logloss: 0.63722
[1468]	valid_0's multi_logloss: 0.637507
[1469]	valid_0's multi_logloss: 0.636766
[1470]	valid_0's multi_logloss: 0.637027
[1471]	valid_0's multi_logloss: 0.636287
[1472]	valid_0's multi_logloss: 0.636581
[1473]	valid_0's 

[1650]	valid_0's multi_logloss: 0.585378
[1651]	valid_0's multi_logloss: 0.584727
[1652]	valid_0's multi_logloss: 0.584078
[1653]	valid_0's multi_logloss: 0.583432
[1654]	valid_0's multi_logloss: 0.583713
[1655]	valid_0's multi_logloss: 0.584012
[1656]	valid_0's multi_logloss: 0.583363
[1657]	valid_0's multi_logloss: 0.583716
[1658]	valid_0's multi_logloss: 0.583948
[1659]	valid_0's multi_logloss: 0.583299
[1660]	valid_0's multi_logloss: 0.583587
[1661]	valid_0's multi_logloss: 0.582938
[1662]	valid_0's multi_logloss: 0.58229
[1663]	valid_0's multi_logloss: 0.582708
[1664]	valid_0's multi_logloss: 0.582061
[1665]	valid_0's multi_logloss: 0.582314
[1666]	valid_0's multi_logloss: 0.582638
[1667]	valid_0's multi_logloss: 0.583068
[1668]	valid_0's multi_logloss: 0.58242
[1669]	valid_0's multi_logloss: 0.582608
[1670]	valid_0's multi_logloss: 0.581961
[1671]	valid_0's multi_logloss: 0.582225
[1672]	valid_0's multi_logloss: 0.582498
[1673]	valid_0's multi_logloss: 0.582861
[1674]	valid_0's m

In [ ]:
imp = pd.DataFrame({'feature': train_df.columns[:-1], 'importance': clf.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh')

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv(submission_file)

# Hyper Parameter tuning _ optuna

In [ ]:
import optuna
# 1. Define an objective function to be maximized.

# dtrain = lgb.Dataset(data = train_df, label = train_df['class'])
y = train_df['class']
X = train_df.drop('class', axis = 1)
X_trn, X_val, y_trn, y_val = train_test_split(X, y, test_size=.1, random_state=seed)
X_trn.shape, X_val.shape, y_trn.shape, y_val.shape

In [ ]:
trn_hp = X_trn.copy()
trn_hp['class'] = y_trn
trn_hp


In [ ]:
trn_hp['class'].value_counts()

In [ ]:
def objective(trial):
    
    dtrain = lgb.Dataset(trn_hp, trn_hp['class'])
    
    # 2. Suggest values of the hyperparameters using a trial object.
    param = {
        #'objective': 'multiclass',
        #'metric': 'multiclass_logloss',
        'verbosity': -1,
        'boosting_type':  'dart',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 200),
        'max_depth' : trial.suggest_int('min_child_samples', 5, 50)
    }
    gbm = lgb.train(param,dtrain)
    
    p_val = gbm.predict(X_val)
    print(p_val.shape)
    
    
    return accuracy_score(y_val, p_val)



In [ ]:
# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2)